# Preprocessing of FIFA 18 - Dataset for Training of Player Classifier
## Import of complete dataset and rough sorting out of unrelevant dimensions
In here, we want to complete the preprocessing for a player classifier, that will take a couple of player attributes and suggests a position the player should take. 
First we will sort out dimensions that obviously do not have a connection to the strength of a player on a position, e.g. club logo, league, shirt number, ...

Our basis is a dataset that comprises 185(!) different attributes for each player! We will reduce it to 35 that might have an impact on the best position of a single player. Moreover we suspect, that the data for players who play in tin pot leagues might be quite bad, so we will keep about 4500 of 17000 player by selecting those who play in famous european leagues.

We start with rough filtering:

In [1]:
#Pandas is used for data wrangling purpose
import pandas as pd
df = pd.read_csv('data/complete.csv')

#DROPPING LISTS DUE TO NO MEANING FOR CLASSIFICATION TASK (e.g. club-logo, number on shirt ,...)
dataset=df.iloc[:,:-91]
to_drop = ['international_reputation', 'potential','stamina','strength','skill_moves', 'weak_foot', 'work_rate_att', 'work_rate_def', 'preferred_foot','gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'ID','birth_date','special','photo','eur_value','eur_wage','eur_release_clause', 'name','club_logo','body_type', 'real_face','flag','nationality']
for i in to_drop:
    dataset=dataset.drop(i,axis=1)

#DROP PLAYERS OF TIN POT LEAGUES (data-quality might be weird here...)
dataset = dataset[df['league'].isin(['Spanish Primera División', 'French Ligue 1', 'German Bundesliga','English Premier League', 'Italian Serie A', 'Turkish Süper Lig','Portuguese Primeira Liga','German 2. Bundesliga'])]
dataset = dataset.dropna(subset=['league'])
dataset = dataset.reset_index()

#EXTRACT THE ONE POSIION WITH BEST RATING AND STORE IN NEW COLUMN 'BestPos':
dataset['BestPos'] = dataset[['rs', 'rw', 'rf', 'ram', 'rcm', 'rm', 'rdm', 'rcb', 'rb', 'rwb', 'st', 'lw', 'cf', 'cam', 'cm', 'lm', 'cdm', 'cb', 'lb', 'lwb', 'ls', 'lf', 'lam', 'lcm', 'ldm', 'lcb', 'gk']].idxmax(axis=1)




#MISCELLANIOUS STEPS USED IN THIS SECTION

#a) look up all leagues in dataset:
#leagues = pd.unique(dataset['league'])

#b) list of column names:
#list(dataset.columns.values)




## Mapping of best positions onto four classes
As we are looking for a first impression of how strong a classifier can be on the dataset, we will aggregate the detailed positions to less classes. E.g. 'right storm', 'left storm', 'storm', central front',... is mapped to just 'storm'.
We are creating classes 'goalkeeper', 'defense', 'middle', 'storm':


In [2]:
#THIS DICT CONTAINS THE MAPPING RULES
position_mapping = {'rs':'Sturm',
                    'rw':'Sturm',
                    'rf':'Sturm',
                    'ram':'Mittelfeld',
                    'rcm':'Mittelfeld',
                    'rm':'Mittelfeld',
                    'rdm':'Mittelfeld',
                    'rcb':'Verteidigung',
                    'rb':'Verteidigung',
                    'rwb':'Verteidigung',
                    'st':'Sturm',
                    'lw':'Sturm',
                    'cf':'Sturm',
                    'cam':'Mittelfeld',
                    'cm':'Mittelfeld',
                    'lm':'Mittelfeld',
                    'cdm':'Mittelfeld',
                    'cb':'Verteidigung',
                    'lb':'Verteidigung',
                    'lwb':'Verteidigung',
                    'ls':'Sturm',
                    'lf':'Sturm',
                    'lam':'Mittelfeld',
                    'lcm':'Mittelfeld',
                    'ldm':'Mittelfeld',
                    'lcb':'Verteidigung',
                    'gk':'Torwart'}


#MAPPING FUNCTION TAKES VALUES OF 'BestPos' AND RETURNS VALUES OF MAPPING, e.g. extract_position(rcm) = 'Mittelfeld',
#WHERE rcm MEANS RIGHT CENTRAL MIDDLEFIELD
def map_position(x):
    for i in position_mapping:
        if x == i:
            return position_mapping[i]

#APPLICATION OF map_position 
dataset['BestPos'] = dataset['BestPos'].apply(map_position)


#SPLIT X INTO TABLE WITH PLAYER INFORMATION (LIKE NAME, CLUB, LEAGUE, AGE) AND INDEPENDENT VARIABLES FOR TRAINING OF CLASSIFIER
player_info= dataset.iloc[:,:5]
X = dataset.iloc[:,5:-29]
#EXTRACT DEPENDENT VARIABLE
y=dataset.iloc[:,-1]
y = pd.DataFrame(y)
y.columns = ['Position']


## Export

In [3]:
#WRITE X,y AND player_info TO CSV-FILES
X.to_csv('X.csv')
y.to_csv('y.csv')
player_info.to_csv('player_info.csv')

